<a href="https://colab.research.google.com/github/zktuong/dandelion/blob/master/container/dandelion_singularity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to `dandelion`
<img src='https://github.com/zktuong/dandelion/blob/master/docs/notebooks/img/dandelion_logo_illustration.png?raw=true'/>

Welcome to the Google Colab notebook example/tutorial for Dandelion! In this notebook, we will run through an example workflow for 1) pre-processing, 2) post-processing and 3) pseudobulk V(D)J trajectory.

## Preprocessing - V(D)J reannotation

We will try out the full reannotation workflow in this part of the colab notebook. To do that, we need to set up the other non-python dependencies. This is normally taken care of by the [singularity container](https://sc-dandelion.readthedocs.io/en/latest/notebooks/Q1-singularity-preprocessing.html) but it's very difficult to set it up on Google Colab architecture, so we will continue manually.

In [ ]:
!pip install -q condacolab
import condacolab
# Run this block by itself first.
# This will lead to a kernel restart, so don't be alarm by the session crashing.
condacolab.install()

In [ ]:
# download reannotation dependencies
!mamba install -y -qq -c bioconda -c conda-forge igblast blast r-optparse r-alakazam r-tigger r-airr r-shazam
# obtain the reference databases
!git clone -q https://github.com/zktuong/dandelion.git
# set paths to databases
import os
os.environ["IGDATA"] = "/content/dandelion/container/database/igblast/"
os.environ["GERMLINE"] = "/content/dandelion/container/database/germlines/"
os.environ["BLASTDB"] = "/content/dandelion/container/database/blast/"

In [ ]:
# install dandelion and scanpy
!pip install -q sc-dandelion scanpy[leiden] networkx==2.7

In [ ]:
# import and show versions
import dandelion as ddl

ddl.logging.print_header()

### Download example datasets from 10X

We will download example datasets freely available from 10X Genomics. To keep this part of the tutorial short, we will trim to just 1000 contigs in the files in the second block but definitely do try it again later on the full datasets!

In [ ]:
%%shell
if [ ! -d "/content/dandelion_tutorial/vdj_v1_hs_pbmc3" ]; then mkdir -p /content/dandelion_tutorial/vdj_v1_hs_pbmc3; fi
# cd /content/dandelion_tutorial/vdj_v1_hs_pbmc3 && wget -q -O filtered_feature_bc_matrix.h5 https://cf.10xgenomics.com/samples/cell-vdj/3.1.0/vdj_v1_hs_pbmc3/vdj_v1_hs_pbmc3_filtered_feature_bc_matrix.h5;
cd /content/dandelion_tutorial/vdj_v1_hs_pbmc3 && wget -q -O filtered_contig_annotations.csv https://cf.10xgenomics.com/samples/cell-vdj/3.1.0/vdj_v1_hs_pbmc3/vdj_v1_hs_pbmc3_b_filtered_contig_annotations.csv;
cd /content/dandelion_tutorial/vdj_v1_hs_pbmc3 && wget -q -O filtered_contig.fasta https://cf.10xgenomics.com/samples/cell-vdj/3.1.0/vdj_v1_hs_pbmc3/vdj_v1_hs_pbmc3_b_filtered_contig.fasta;

if [ ! -d "/content/dandelion_tutorial/vdj_nextgem_hs_pbmc3" ]; then mkdir -p /content/dandelion_tutorial/vdj_nextgem_hs_pbmc3; fi
# cd /content/dandelion_tutorial/vdj_nextgem_hs_pbmc3 && wget -q -O filtered_feature_bc_matrix.h5 https://cf.10xgenomics.com/samples/cell-vdj/3.1.0/vdj_nextgem_hs_pbmc3/vdj_nextgem_hs_pbmc3_filtered_feature_bc_matrix.h5;
cd /content/dandelion_tutorial/vdj_nextgem_hs_pbmc3 && wget -q -O filtered_contig_annotations.csv https://cf.10xgenomics.com/samples/cell-vdj/3.1.0/vdj_nextgem_hs_pbmc3/vdj_nextgem_hs_pbmc3_b_filtered_contig_annotations.csv;
cd /content/dandelion_tutorial/vdj_nextgem_hs_pbmc3 && wget -q -O filtered_contig.fasta https://cf.10xgenomics.com/samples/cell-vdj/3.1.0/vdj_nextgem_hs_pbmc3/vdj_nextgem_hs_pbmc3_b_filtered_contig.fasta;

if [ ! -d "/content/dandelion_tutorial/sc5p_v2_hs_PBMC_10k" ]; then mkdir -p /content/dandelion_tutorial/sc5p_v2_hs_PBMC_10k; fi
# cd /content/dandelion_tutorial/sc5p_v2_hs_PBMC_10k && wget -q -O filtered_feature_bc_matrix.h5 https://cf.10xgenomics.com/samples/cell-vdj/4.0.0/sc5p_v2_hs_PBMC_10k/sc5p_v2_hs_PBMC_10k_filtered_feature_bc_matrix.h5;
cd /content/dandelion_tutorial/sc5p_v2_hs_PBMC_10k && wget -q -O filtered_contig_annotations.csv https://cf.10xgenomics.com/samples/cell-vdj/4.0.0/sc5p_v2_hs_PBMC_10k/sc5p_v2_hs_PBMC_10k_b_filtered_contig_annotations.csv;
cd /content/dandelion_tutorial/sc5p_v2_hs_PBMC_10k && wget -q -O filtered_contig.fasta https://cf.10xgenomics.com/samples/cell-vdj/4.0.0/sc5p_v2_hs_PBMC_10k/sc5p_v2_hs_PBMC_10k_b_filtered_contig.fasta;

if [ ! -d "/content/dandelion_tutorial/sc5p_v2_hs_PBMC_1k" ]; then mkdir -p /content/dandelion_tutorial/sc5p_v2_hs_PBMC_1k; fi
# cd /content/dandelion_tutorial/sc5p_v2_hs_PBMC_1k && wget -q -O filtered_feature_bc_matrix.h5 wget https://cf.10xgenomics.com/samples/cell-vdj/4.0.0/sc5p_v2_hs_PBMC_1k/sc5p_v2_hs_PBMC_1k_filtered_feature_bc_matrix.h5;
cd /content/dandelion_tutorial/sc5p_v2_hs_PBMC_1k && wget -q -O filtered_contig_annotations.csv wget https://cf.10xgenomics.com/samples/cell-vdj/4.0.0/sc5p_v2_hs_PBMC_1k/sc5p_v2_hs_PBMC_1k_b_filtered_contig_annotations.csv;
cd /content/dandelion_tutorial/sc5p_v2_hs_PBMC_1k && wget -q -O filtered_contig.fasta https://cf.10xgenomics.com/samples/cell-vdj/4.0.0/sc5p_v2_hs_PBMC_1k/sc5p_v2_hs_PBMC_1k_b_filtered_contig.fasta;

In [ ]:
%%shell
# trim to just 1000 contigs to make this part of the tutorial shorter
cd /content/dandelion_tutorial/vdj_v1_hs_pbmc3 && head -1000 filtered_contig.fasta > filtered_1k_contig.fasta && sed -i '1,+999d' filtered_contig.fasta && head -1001 filtered_contig_annotations.csv > filtered_1k_contig_annotations.csv && sed -i '1,+1000d' filtered_contig_annotations.csv
cd /content/dandelion_tutorial/vdj_nextgem_hs_pbmc3 && head -1000 filtered_contig.fasta > filtered_1k_contig.fasta && sed -i '1,+999d' filtered_contig.fasta && head -1001 filtered_contig_annotations.csv > filtered_1k_contig_annotations.csv && sed -i '1,+1000d' filtered_contig_annotations.csv
cd /content/dandelion_tutorial/sc5p_v2_hs_PBMC_10k && head -1000 filtered_contig.fasta > filtered_1k_contig.fasta && sed -i '1,+999d' filtered_contig.fasta && head -1001 filtered_contig_annotations.csv > filtered_1k_contig_annotations.csv && sed -i '1,+1000d' filtered_contig_annotations.csv
cd /content/dandelion_tutorial/sc5p_v2_hs_PBMC_1k && head -1000 filtered_contig.fasta > filtered_1k_contig.fasta && sed -i '1,+999d' filtered_contig.fasta && head -1001 filtered_contig_annotations.csv > filtered_1k_contig_annotations.csv && sed -i '1,+1000d' filtered_contig_annotations.csv

## Preprocessing 10X data

Just like the [original tutorial](https://sc-dandelion.readthedocs.io/en/latest/notebooks/1_dandelion_preprocessing-10x_data.html), we will format the files, reannotate with `igblastn`, reassign the V gene alleles, assign the isotype calls and quantify mutations.

In [ ]:
os.chdir("/content/dandelion_tutorial")
samples = [
    "sc5p_v2_hs_PBMC_1k",
    "sc5p_v2_hs_PBMC_10k",
    "vdj_v1_hs_pbmc3",
    "vdj_nextgem_hs_pbmc3",
]
filename_prefixes = ["filtered_1k" for s in samples]

In [ ]:
ddl.pp.format_fastas(samples, prefix=samples, filename_prefix=filename_prefixes)

In [ ]:
ddl.pp.reannotate_genes(samples, filename_prefix=filename_prefixes)


In [ ]:
# reassigning alleles on the first set of samples
ddl.pp.reassign_alleles(
    samples[:2],
    combined_folder="tutorial_scgp1",
    filename_prefix=filename_prefixes,
)

In [ ]:
# reassigning alleles on the second set of samples
ddl.pp.reassign_alleles(
    samples[2:],
    combined_folder="tutorial_scgp2",
    filename_prefix=filename_prefixes,
)

In [ ]:
ddl.pp.assign_isotypes(samples, filename_prefix=filename_prefixes)


In [ ]:
from tqdm import tqdm

# quantify mutations
for s in tqdm(samples, desc="Basic mutational load analysis "):
    filePath = s + "/dandelion/filtered_1k_contig_dandelion.tsv"
    ddl.pp.quantify_mutations(filePath)

And now we have a processed output (`filtered_1k_contig_dandelion.tsv`) in each folder that can be used for downstream exploration.

# Post-processing - V(D)J analysis

We will now switch to the post-processing tutorial. Let's follow the rest of the [original tutorial](https://sc-dandelion.readthedocs.io/en/latest/notebooks/Q2-analysis.html).

In [ ]:
import dandelion as ddl
import scanpy as sc
import warnings
import os

warnings.filterwarnings("ignore")
ddl.logging.print_header()

In [ ]:
sc.logging.print_header()


Let’s run through some of what `Dandelion` can do in terms of analysis. In order to kickstart this tutorial, we prepared GEX and V(D)J objects with four demo 10X samples parsed for your convenience. You can download them from the ftp site as per below. The GEX has had some basic `Scanpy` analysis performed on it, all the way up to making a UMAP and calling Leiden clusters. The VDJ is just four calls of `ddl.read_10x_airr()`, with the individual objects saved to a list, which was then passed to `ddl.concat()` to get a combined `Dandelion` object. No analysis was done on this combined object.

In [ ]:
if not os.path.exists("demo-gex.h5ad"):
    os.system("wget ftp://ftp.sanger.ac.uk/pub/users/kp9/demo-gex.h5ad")

if not os.path.exists("demo-vdj.h5ddl"):
    os.system("wget ftp://ftp.sanger.ac.uk/pub/users/kp9/demo-vdj.h5ddl")


Let’s import the objects. `Dandelion`’s h5ddl files can be read via `ddl.read_h5ddl()`.

In [ ]:
adata = sc.read("demo-gex.h5ad")
vdj = ddl.read_h5ddl("demo-vdj.h5ddl")


At this point you’re probably wondering why there’s a separate Dandelion object. The reason is AIRR compliance. Some of the AIRR columns have more complex typing than what Scanpy can currently support within its objects. However, it’s quite straightforward to link up a Scanpy object with a `Dandelion` one.

In [ ]:
vdj, adata = ddl.pp.check_contigs(vdj, adata)


This filters the contigs and synchronises relevant information between the objects. Once linked up like this, any new information can be copied over from the Dandelion object via `ddl.tl.transfer()`.

For now, let’s take a look at the chain status (as gotten from the Dandelion object) and known BCR marker expression.

In [ ]:
sc.pl.umap(adata, color=["IGHM", "JCHAIN", "chain_status"])

Under the hood, the `Dandelion` object is essentially two data frames. .data holds the AIRR-compliant contig space table, while `.metadata` is an `.obs` equivalent that parses the contig information to a cell level and can be easily integrated with a `Scanpy` object. There are also `ddl.to_scirpy()` and `ddl.from_scirpy()` for interoperability with `Scirpy`, as explored in a notebook in the extended tutorial.

In [ ]:
vdj


Now that we’ve got the gist of basic handling of the `Dandelion` object, let’s use it for some analysis!

A core element of V(D)J analysis is clonotype calling, roughly equivalent to clustering cells in GEX processing. `Dandelion` requires the clones it calls to have identical V and J genes, along with no more than 15% mismatches in the CDR3 sequences (common practice in BCR analysis).

In [ ]:
ddl.tl.find_clones(vdj)


We can compute a graph based on Levenshtein distance of the complete contig sequence. A NetworkX representation of it is now saved in `vdj.graph`.

In [ ]:
ddl.tl.generate_network(vdj)


`graph-tool` is very difficult to install on Google Colab so we will skip that for now.

Since we now know what our clonotype calls are, we can quantify clonal expansion. It’s possible to cap this at a desired maximum clonotype size.

In [ ]:
ddl.tl.clone_size(vdj)
ddl.tl.clone_size(vdj, max_size=3)

Now that our Dandelion object has analysis information inside it, we can copy it over to the Scanpy object to have access to it there. The graph gets turned into the Scanpy standard forms of `.obsp['vdj_distances']` and `.obsp['vdj_connectivites']` for potential downstream use.

In [ ]:
ddl.tl.transfer(adata, vdj)


Let’s take a look at what we made!

In [ ]:
ddl.pl.clone_network(adata, color="clone_id_size")
sc.pl.umap(adata, color="clone_id_size")

Wait, why are we seeing some clone size 0 in the plots? Orphan chains.

In [ ]:
ddl.pl.clone_network(adata, color="clone_id_size_max_3", na_in_legend=False)
sc.pl.umap(adata, color="clone_id_size_max_3", na_in_legend=False)

Dandelion comes with a number of plotting functions for your convenience. However, those functions tend to operate best without the Scanpy plotting defaults in place. You can reset Matplotlib’s configuration prior to using them.

In [ ]:
import matplotlib as mpl
mpl.rcParams.update(mpl.rcParamsDefault)
%matplotlib inline

We’ve got bar plots.

In [ ]:
ddl.pl.barplot(
    vdj[vdj.metadata.isotype_status != "Multi"],  # remove multi from the plots
    color="v_call_genotyped_VDJ",
    xtick_fontsize=5,
)

All of the plotting functions have a number of parameters that can be fiddled with for desired visualisation outcomes. For example, let’s disable automatic descending sorting, show counts rather than proportions, and change the palette.

In [ ]:
ddl.pl.barplot(
    vdj[vdj.metadata.isotype_status != "Multi"],
    color="v_call_genotyped_VDJ",
    normalize=False,
    sort_descending=None,
    palette="tab20",
    xtick_fontsize=5,
)

We’ve got stacked bar plots.

In [ ]:
ddl.pl.stackedbarplot(
    vdj[vdj.metadata.isotype_status != "Multi"],
    color="isotype_status",
    groupby="locus_status",
    xtick_rotation=0,
    figsize=(4, 3),
)

These can be normalised to add up to 1 for each column.

In [ ]:
ddl.pl.stackedbarplot(
    vdj[vdj.metadata.isotype_status != "Multi"],
    color="v_call_genotyped_VDJ",
    groupby="isotype_status",
    normalize=True,
    xtick_fontsize=5,
)

We’ve also got a spectratype plot, which shows the distribution of the CDR3 length for the various contigs.

In [ ]:
ddl.pl.spectratype(
    vdj[vdj.metadata.isotype_status != "Multi"],
    color="junction_length",
    groupby="c_call",
    locus="IGH",
    width=2.3,
)

Another common V(D)J analysis request is to examine the distribution of shared clonotypes between cells of different metadata groups. Dandelion can do this as a circos plot.

In [ ]:
ddl.tl.clone_overlap(adata, groupby="leiden", weighted_overlap=True)
ddl.pl.clone_overlap(adata, groupby="leiden", weighted_overlap=True)

There’s also a heatmap on offer.

In [ ]:
ddl.pl.clone_overlap(
    adata,
    groupby="leiden",
    weighted_overlap=True,
    as_heatmap=True,
    # seaborn clustermap kwargs
    cmap="Blues",
    annot=True,
    figsize=(8, 8),
    annot_kws={"size": 10},
)

Save the objects, like so.

In [ ]:
adata.write("demo-gex-processed.h5ad")
vdj.write("demo-vdj-processed.h5ddl")


# V(D)J pseudobulk feature space

In [ ]:
import dandelion as ddl
import pandas as pd
import scanpy as sc
import numpy as np
import warnings
import os

warnings.filterwarnings("ignore")

This notebook makes use of `Milopy` and `Palantir`, two packages that are not formally `Dandelion`’s dependencies. V(D)J feature space applications are open-ended, this is just one of them. Be sure to install the packages beforehand if you want to follow along.

In [ ]:
!pip install git+https://github.com/emdann/milopy.git palantir -qq

In [ ]:
import milopy.core as milo
import palantir

#required because of Palantir
%matplotlib inline

sc.settings.set_figure_params(dpi=80)

We’ve prepared a demo object based on the TCR trajectory shown in the manuscript for you to use here. It’s had some analysis done on the GEX, and has Dandelion-derived contig information merged into it. You can download it from the ftp site as per below.

It’s possible to use V(D)J information that comes from other sources than Dandelion processing, e.g. the pseudobulking will work with Scirpy output. The functions are just calibrated to work with Dandelion’s structure by default.

In [ ]:
if not os.path.exists("demo-pseudobulk.h5ad"):
    os.system("wget ftp://ftp.sanger.ac.uk/pub/users/kp9/demo-pseudobulk.h5ad")


Prior to performing the pseudobulking, it is recommended to run `ddl.tl.setup_vdj_pseudobulk()`. This will subset the object to just cells with paired chains, and prepare appropriately named and formatted columns for the pseudobulking function to use as defaults.

If working with non-Dandelion V(D)J processing, subset your cells to ones with at least a full pair of chains, and ensure that you have four columns in place which contain the V(D)J calls for both of the identified primary chains. Scirpy stores this information natively.

If you are wanting to include D calls (disabled by default), the recommendation is to subset to only cells/contigs with d_call annotated otherwise the separation could be unreliable (due to missing d_call because of technical reasons rather than biology). Please look at the options for `ddl.tl.setup_vdj_pseudobulk()` carefully to tailor to your use case.

We will proceed with the default settings, which is to only consider the primary V and J calls (productive and highest UMI).

In [ ]:
adata = sc.read("demo-pseudobulk.h5ad")
adata = ddl.tl.setup_vdj_pseudobulk(adata)


We’re going to be using Milopy to create pseudobulks. Construct a neighbour graph with many neighbours, following Milopy protocol, and then sample representative neighbourhoods from the object. This saves a cell-by-pseudobulk matrix into `adata.obsm["nhoods"]`. Use this graph to generate a UMAP as well.

In [ ]:
sc.pp.neighbors(adata, use_rep="X_scvi", n_neighbors=50)
milo.make_nhoods(adata)
sc.tl.umap(adata)

Now we are armed with everything we need to construct the V(D)J feature space. Pseudobulks can be defined either via passing a list of `.obs` metadata columns, the unique values of the combination of which will serve as individual pseudobulks (via `obs_to_bulk`), or via an explicit cell-by-pseudobulk matrix (via `pbs`). Milopy created one of those for us, so we can use that as input.

The cell type annotation lives in `.obs["anno_lvl_2_final_clean"]`. Let’s tell the function that we want to take the most common value per pseudobulk with us to the new V(D)J feature space object.

For non-Dandelion V(D)J processing, use the cols argument to specify which .obs columns contain the V(D)J calls for the identified primary chains. For Scirpy, this would mean specifying e.g. `cols = ['IR_VDJ_1_v_gene', 'IR_VDJ_1_j_gene', 'IR_VJ_1_v_gene', 'IR_VJ_1_j_gene']`.

In [ ]:
pb_adata = ddl.tl.vdj_pseudobulk(
    adata, pbs=adata.obsm["nhoods"], obs_to_take="anno_lvl_2_final_clean"
)

The new object has pseudobulks as observations, and the unique encountered V(D)J genes as the features. We can see the per-pseudobulk annotation, and `.uns["pseudobulk_assigments"]`. In our case it’s just a copy of the `pbs` argument, but if we were to go for `obs_to_bulk` this would be a cells by pseudobulks matrix capturing the assignment of the original cells.

In [ ]:
pb_adata


Now that we have our V(D)J feature space pseudobulk object, we can do things with it. Let’s run a PCA on it. The development trajectory is very nicely captured in the first two PC dimensions.

In [ ]:
sc.tl.pca(pb_adata)
sc.pl.pca(pb_adata, color="anno_lvl_2_final_clean")


Let’s define the start of our trajectory as the right-most cell, the CD4 terminal state as the bottom-most cell, and the CD8 terminal state as the top-most cell. We can then follow Palantir protocol to generate a diffusion map and run pseudotime. Once done, we rename the terminal states to be more informative.

In [ ]:
rootcell = np.argmax(pb_adata.obsm["X_pca"][:, 0])
terminal_states = pd.Series(
    ["CD8+T", "CD4+T"],
    index=pb_adata.obs_names[
        [
            np.argmax(pb_adata.obsm["X_pca"][:, 1]),
            np.argmin(pb_adata.obsm["X_pca"][:, 1]),
        ]
    ],
)

# Run diffusion maps
pca_projections = pd.DataFrame(pb_adata.obsm["X_pca"], index=pb_adata.obs_names)
dm_res = palantir.utils.run_diffusion_maps(pca_projections, n_components=5)
ms_data = palantir.utils.determine_multiscale_space(dm_res)

pr_res = palantir.core.run_palantir(
    ms_data,
    pb_adata.obs_names[rootcell],
    num_waypoints=500,
    terminal_states=terminal_states.index,
)

pr_res.branch_probs.columns = terminal_states[pr_res.branch_probs.columns]

We can easily transfer the inferred pseudotime and branching probabilities to the pseudobulk object with the aid of a helper function.

In [ ]:
pb_adata = ddl.tl.pseudotime_transfer(pb_adata, pr_res)
sc.pl.pca(
    pb_adata,
    color=["pseudotime", "prob_CD4+T", "prob_CD8+T"],
    color_map="coolwarm",
)

We can project back our findings to the original cell space object via another helper function. This will remove any cells not in any of the pseudobulks. In the event of a cell belonging to multiple pseudobulks, the cell’s pseudotime will be the average of the pseudobulks weighted by the inverse of the pseudobulk size.

In [ ]:
bdata = ddl.tl.project_pseudotime_to_cell(
    adata, pb_adata, terminal_states.values
)
sc.pl.umap(bdata, color=["anno_lvl_2_final_clean"])
sc.pl.umap(
    bdata,
    color=["pseudotime", "prob_CD4+T", "prob_CD8+T"],
    color_map="coolwarm",
)

For more in-depth tutorial and examples of the workflow, please visit `dandelion`'s [documentation](https://sc-dandelion.readthedocs.io/) and [github repository](https://github.com/zktuong/dandelion).